# <center> Homework Week 4 </center

In [1]:
import numpy as np
from collections import defaultdict
from functions import read_q
from itertools import permutations
from functions_week3 import overlap

[[0 4 2 4 8]
 [4 0 4 2 8]
 [2 4 0 4 8]
 [4 2 4 0 8]
 [8 8 8 8 8]]


<b>Question 1</b><br>
In a practical, we saw the scs function (copied below along with overlap) for finding the shortest common superstring of a set of strings.<br>
<code>
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match<br>
import itertools
def scs(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    shortest_sup = None
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_sup is None or len(sup) < len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
    return shortest_sup  # return shortest
</code>
<br>
It's possible for there to be multiple different shortest common superstrings for the same set of input strings. Consider the input strings ABC, BCA, CAB. One shortest common superstring is ABCAB but another is BCABC and another is CABCA.



What is the length of the shortest common superstring of the following strings?

CCT, CTT, TGC, TGG, GAT, ATT

In [2]:
def scs(str_list,k):
    reference_string=''.join(str_list)
    ss_set=[]
    count=0
    for strings in permutations(str_list):
        count_overlap=0
        final_str=strings[0]  #Basically, at the end, we are appending the overlap value of the second string to the final_str. Hence, if any 'strings' is not considered as the second string in overlap() e.g. first string in the permutation set, it will not be updated in the final string 
        for i in range(len(str_list)-k):
            oval=overlap(strings[i],strings[i+1],k)
            final_str+=strings[i+1][oval:]
            count_overlap+=1
        ss_set.append(final_str)
        if len(final_str)<len(reference_string):
            reference_string=final_str
        count+=1
    count_incidence=0
    for ss in ss_set:
        if len(ss)==len(reference_string):
            count_incidence+=1
    return reference_string,count_incidence,ss_set

In [3]:
final_str, count, sset=scs(('CCT','CTT','TGC','TGG','GAT', 'ATT'),1)

In [4]:
print('The length of the final SCS string is:',len(final_str))

The length of the final SCS string is: 11


<b>Question 2: </b><br>
How many different shortest common superstrings are there for the input strings given in the previous question?
    

In [5]:
print(f'There are {count} different shortest common superstrings')

There are 4 different shortest common superstrings


<b>Question 3:</b><br>
Download this FASTQ file containing synthetic sequencing reads from a mystery virus:

https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ads1_week4_reads.fq

All the reads are the same length (100 bases) and are exact copies of substrings from the forward strand of the virus genome.  You don't have to worry about sequencing errors, ploidy, or reads coming from the reverse strand.

Assemble these reads using one of the approaches discussed, such as greedy shortest common superstring.  Since there are many reads, you might consider ways to make the algorithm faster, such as the one discussed in the programming assignment in the previous module.

How many As are there in the full, assembled genome?

Hint: the virus genome you are assembling is exactly 15,894 bases long

In [6]:
def fast_overlap(reads,kmer=6,minlength=3):
    count=0
    max_olen=0
    max_a,max_b='',''
    dict_kmers=defaultdict(set)
    for read in reads:
        for t in range(len(read)-kmer+1):
            dict_kmers[read[t:t+kmer]].add(read)
 #This is a dictionary with key=kmer and value=entire read
    overlap_dict=defaultdict(set)
    for read in reads:
        suffs=read[-kmer:] #Get the last kmer/ suffix of length k from the read
        read_suffs=dict_kmers[suffs] #Get all the reads that have this suffix somewhere in their list
        for suff in read_suffs: 
            if read!=suff:#If the original read is not equal to the current read selected:
                olen=overlap(read,suff,minlength)
                if olen>max_olen:
                    max_a=read
                    max_b=suff
                    max_olen=olen
    return max_a, max_b, max_olen

In [7]:
def fast_greedy(str_list,k=3):
    a,b,olen=fast_overlap(str_list,k)
    i=0
    while olen>0:
#         print('\ri {}'.format(i), end="") #Just to count how many iterations it goes through
        str_list.remove(a)
        str_list.remove(b)
        str_list.append(a+b[olen:])
        a,b,olen=fast_overlap(str_list,k)
        i+=1
    if len(str_list)==1:
        return str_list
    else:
        return ''.join(str_list)

In [8]:
seq, _=read_q('ads1_week4_reads.fq')

The question does not let us know how many k-mers to take but does tell us what the expected length of the final string is going to be. Hence, we start from the highest k-mer (This is a shortcut because lower the k-mer, larger number of patches of the string and larger the number of comparisons) and keep checking the length of the final assembly

In [9]:
import time

In [10]:
start_time=time.time()
l=0 
k=100 #Maximum k for k-mer since that is the entire length of a single read
while l!=15894:
    x=fast_greedy(seq,k)
    l=len(x[0])
    k=k-1
    if k<=0:
        print('Didnt get correct l')
        break
print('\nNo. of k-mers:',k+1)
print('Length of final assembly',len(x[0]))
end_time=time.time()


No. of k-mers: 52
Length of final assembly 15894


In [11]:
print(f"There are {x[0].count('A')} A's in the fully assembled genome")

There are 4633 A's in the fully assembled genome


In [12]:
print(f"There are {x[0].count('T')} T's in the fully assembled genome")

There are 3723 T's in the fully assembled genome


In [13]:
print('Time taken to complete fast_greedy() loop', end_time-start_time, 'seconds')

Time taken to complete fast_greedy() loop 26.13089156150818 seconds


### Diversion

<b> Comparison between fast_greedy() and normal greedy_scs() for Q5 and Q6 </b>

In [14]:
def get_maximal_overlap(str_list,k): #implementation based on lecture. Need to try dictionary approach
    best_olen=0
    best_a,best_b='',''
    for a,b in permutations(str_list,2):
        oval=overlap(a,b,k)
        if oval>best_olen:
            best_a, best_b, best_olen= a,b,oval
    return best_a, best_b, best_olen

In [15]:
def greedy_scs(str_list,k):
    a,b,olen=get_maximal_overlap(str_list,k)
    while olen>0:
        str_list.remove(a)
        str_list.remove(b)
        str_list.append(a+b[olen:])
        a,b,olen=get_maximal_overlap(str_list,k)
    if len(str_list)>1:
        return str_list
    else:
        return ''.join(str_list)

In [18]:
seq, _=read_q('ads1_week4_reads.fq') #Since seq is modified after scs or greedy_scs, we need  to reset this

In [19]:
start_time1=time.time()
l=0 
k=100 #Maximum k for k-mer since that is the entire length of a single read
while l!=15894:
    x=greedy_scs(seq,k)
    l=len(x[0])
    k=k-1
    if k<=0:
        print('Didnt get correct l')
        break
print('\n')
print(k+1)
print('\n')
print(x)
end_time1=time.time()

Didnt get correct l


1


ACCAAACAAAGTTGGGTAAGGATAGATCAATCAATGATCATATTCTAGTACACTTAGGATTCAAGATCCTATTATCAGGGACAAGAGCAGGATTAGGGATATCCGAGATGGCCACACTTTTGAGGAGCTTAGCATTGTTCAAAAGAAACAAGGACAAACCACCCATTACATCAGGATCCGGTGGAGCCATCAGAGGAATCAAACACATTATTATAGTACCAATTCCTGGAGATTCCTCAATTACCACTCGATCCAGACTACTGGACCGGTTGGTCAGGTTAATTGGAAACCCGGATGTGAGCGGGCCCAAACTAACAGGGGCACTAATAGGTATATTATCCTTATTTGTGGAGTCTCCAGGTCAATTGATTCAGAGGATCACCGATGACCCTGACGTTAGCATCAGGCTGTTAGAGGTTGTTCAGAGTGACCAGTCACAATCTGGCCTTACCTTCGCATCAAGAGGTACCAACATGGAGGATGAGGCGGACCAATACTTTTCACATGATGATCCAAGCAGTAGTGATCAATCCAGGTCCGGATGGTTCGAGAACAAGGAAATCTCAGATATTGAAGTGCAAGACCCTGAGGGATTCAACATGATTCTGGGTACCATTCTAGCCCAGATCTGGGTCTTGCTCGCAAAGGCGGTTACGGCCCCAGACACGGCAGCTGATTCGGAGCTAAGAAGGTGGATAAAGTACACCCAACAAAGAAGGGTAGTTGGTGAATTTAGATTGGAGAGAAAATGGTTGGATGTGGTGAGGAACAGGATTGCCGAGGACCTCTCTTTACGCCGATTCATGGTGGCTCTAATCCTGGATATCAAGAGGACACCCGGGAACAAACCTAGGATTGCTGAAATGATATGTGACATTGATACATATATCGTAGAGGCAGGATTAGCCAGTTTTATCCTGACTATTAAGTTTGGGATAGAAACTATGTATCCTGCTCTTGGACTGCATGAATTT

In [20]:
print('Time taken to complete fast_greedy() loop', end_time1-start_time1, 'seconds')

Time taken to complete fast_greedy() loop 1822.2057871818542 seconds


The time taken for the fast_greedy() algorithm was 26.13 seconds while the time for the greedy_scs() algorithm was 1822.21 seconds (30 mins). Hence, we see an improvement of almost 70X with the fast_greedy() algorithm